In [20]:
# from google.colab import files

# # Upload the file
# uploaded = files.upload()

In [21]:
!pip install ucimlrepo

In [22]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import SelectKBest

from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [23]:
# # Step 1: Load your dataset
# data = pd.read_csv('heart_attack_risk_dataset.csv')

In [24]:
# # Step 2: Separate features (X) and target (y)
# TARGET_VARIABLE = 'Heart_Attack_Risk'
# X = data.drop(columns=[TARGET_VARIABLE])
# y = data[TARGET_VARIABLE]

In [25]:
# # Step 3: Identify and encode categorical columns

# # Identify categorical
# categorical_cols = X.select_dtypes(include=['object']).columns

# # Step 3: Encode categorical columns
# label_encoders = {}
# for col in categorical_cols:
#     le = LabelEncoder()
#     X[col] = le.fit_transform(X[col])
#     label_encoders[col] = le

# # Encode the target variable as well
# y = LabelEncoder().fit_transform(y)

In [26]:
isolet = fetch_ucirepo(id=54)

X = isolet.data.features
y = isolet.data.targets

In [27]:
X_df = pd.DataFrame(X)
y_df = pd.DataFrame(y)

print("Missing values in X:", X_df.isnull().sum().sum())
print("Missing values in y:", y_df.isnull().sum().sum())

Missing values in X: 0
Missing values in y: 0


In [28]:
print("Minimum value across all features:", X_df.min().min())
print("Maximum value across all features:", X_df.max().max())

Minimum value across all features: -1.0
Maximum value across all features: 1.0


In [29]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_df)

X_scaled_df = pd.DataFrame(X_scaled, columns=X_df.columns)

print(X_scaled_df.min().min())
print(X_scaled_df.max().max())

0.0
1.0000000000000002


In [30]:
# spambase = fetch_ucirepo(id=94)

# X = spambase.data.features
# y = spambase.data.targets

In [31]:
# X_df = pd.DataFrame(X)
# y_df = pd.DataFrame(y)

# print("Missing values in X:", X_df.isnull().sum().sum())
# print("Missing values in y:", y_df.isnull().sum().sum())

In [32]:
# print("Minimum value across all features:", X_df.min().min())
# print("Maximum value across all features:", X_df.max().max())

In [33]:
# scaler = MinMaxScaler()
# X_scaled = scaler.fit_transform(X_df)

# X_scaled_df = pd.DataFrame(X_scaled, columns=X_df.columns)

# print(X_scaled_df.min().min())
# print(X_scaled_df.max().max())

In [34]:
X = X_scaled_df
y = y_df

In [35]:
def get_top_k_features(selector, feature_names, k, algorithm):
  """
  Get the top k features based on their scores from a SelectKBest selector.

  Parameters:
  selector (SelectKBest): Fitted SelectKBest object.
  feature_names (list): List of feature names (columns of X).
  k (int): Number of top features to select.
  algorithm (str): The name of the feature selection algorithm.

  Returns:
  list: Names of the top k features.
  """
  # Retrieve feature scores
  scores = selector.scores_

  # Create a DataFrame for ranked features
  feature_ranking = pd.DataFrame({
    'Feature': feature_names,
    'Score': scores
  }).sort_values(by='Score', ascending=False)

  # Display top-ranked features
  print(f"Feature Rankings using {algorithm}:")
  print(feature_ranking)

  # Return selected top k features
  return feature_ranking.head(k)['Feature'].tolist()

In [36]:
def train_and_fit_model(X_train, X_test, y_train, y_test):
  """
  Builds, trains, and evaluates a Random Forest classification model.

  Parameters:
  ----------
  X_train : pd.DataFrame or np.ndarray
      Feature matrix for training the model.
  X_test : pd.DataFrame or np.ndarray
      Feature matrix for testing the model.
  y_train : pd.Series or np.ndarray
      Target labels for training the model.
  y_test : pd.Series or np.ndarray
      True target labels for testing the model.

  Returns:
  -------
  None
      Prints the model's accuracy and a detailed classification report.
  """
  # Build a simple classification model
  model = RandomForestClassifier(random_state=42)
  model.fit(X_train, y_train)

  # Make predictions
  y_pred = model.predict(X_test)

  # Evaluate the model
  accuracy = accuracy_score(y_test, y_pred)
  print("\nModel Accuracy with Top 10 Features:", accuracy)

  # Detailed performance metrics
  print("\nClassification Report:")
  print(classification_report(y_test, y_pred))

In [37]:
from sklearn.feature_selection import chi2
# Step 4: Apply Chi-Square

# SelectKBest with chi2 evaluates all features
selector = SelectKBest(score_func=chi2, k='all')
selector.fit(X, y)

# Rank the features using Chi-Square algorithm
top_features = get_top_k_features(selector=selector, feature_names=X.columns,
                                  k=20, algorithm="Chi-Square")

# Reduce the dataset to the top 10 features
X_top = X[top_features]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_top, y, test_size=0.2, random_state=42)

# Train and fit random forest classification model based on feature selected
train_and_fit_model(X_train, X_test, y_train, y_test)

Feature Rankings using Chi-Square:
          Feature        Score
418  Attribute419  5041.889698
417  Attribute418  4948.081234
416  Attribute417  4712.209974
415  Attribute416  4316.661306
388  Attribute389  4286.109851
..            ...          ...
601  Attribute602    35.166998
344  Attribute345    32.509363
600  Attribute601    31.842739
343  Attribute344    27.478685
472  Attribute473    14.244934

[617 rows x 2 columns]


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Model Accuracy with Top 10 Features: 0.39294871794871794

Classification Report:
              precision    recall  f1-score   support

         1.0       0.06      0.05      0.05        66
         2.0       0.15      0.16      0.16        50
         3.0       0.82      0.79      0.80        62
         4.0       0.12      0.08      0.10        59
         5.0       0.06      0.05      0.05        64
         6.0       0.78      0.90      0.83        59
         7.0       0.28      0.28      0.28        53
         8.0       0.92      0.81      0.86        70
         9.0       0.09      0.11      0.10        56
        10.0       0.25      0.31      0.28        55
        11.0       0.64      0.47      0.54        78
        12.0       0.06      0.04      0.05        67
        13.0       0.11      0.12      0.11        58
        14.0       0.10      0.11      0.10        66
        15.0       0.15      0.20      0.17        54
        16.0       0.43      0.49      0.46        59

In [38]:
from sklearn.feature_selection import mutual_info_classif
# Step 5: Apply Mutual Information Classification (MIC)

# Wrapping mutual_info_classif with a fixed random_state
mutual_info_classif_with_random_state = lambda X, y: mutual_info_classif(X, y, random_state=42)

# SelectKBest with mutual_info_classif evaluates all features
selector = SelectKBest(score_func=mutual_info_classif_with_random_state, k='all')
selector.fit(X, y)

# Rank the features using Mutual Information Classification algorithm
top_features = get_top_k_features(selector=selector, feature_names=X.columns,
                                  k=20, algorithm="Mutual Information Classification")

# Reduce the dataset to the top 10 features
X_top = X[top_features]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_top, y, test_size=0.2, random_state=42)

# Train and fit random forest classification model based on feature selected
train_and_fit_model(X_train, X_test, y_train, y_test)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Feature Rankings using Mutual Information Classification:
          Feature     Score
461  Attribute462  0.851708
460  Attribute461  0.770072
105  Attribute106  0.754416
104  Attribute105  0.742367
459  Attribute460  0.735609
..            ...       ...
287  Attribute288  0.063881
601  Attribute602  0.061680
582  Attribute583  0.060256
374  Attribute375  0.059147
472  Attribute473  0.038452

[617 rows x 2 columns]


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Model Accuracy with Top 10 Features: 0.642948717948718

Classification Report:
              precision    recall  f1-score   support

         1.0       0.54      0.67      0.60        66
         2.0       0.42      0.56      0.48        50
         3.0       0.79      0.87      0.83        62
         4.0       0.72      0.78      0.75        59
         5.0       0.49      0.31      0.38        64
         6.0       0.32      0.32      0.32        59
         7.0       0.61      0.79      0.69        53
         8.0       0.58      0.54      0.56        70
         9.0       0.50      0.61      0.55        56
        10.0       0.84      0.85      0.85        55
        11.0       0.91      0.86      0.88        78
        12.0       0.69      0.60      0.64        67
        13.0       0.82      0.78      0.80        58
        14.0       0.71      0.68      0.70        66
        15.0       0.71      0.72      0.72        54
        16.0       0.60      0.61      0.61        59
 

In [39]:
from sklearn.feature_selection import mutual_info_regression
# Step 6: Apply Mutual Information Regression (MIR)

# Wrapping mutual_info_regression with a fixed random_state
mutual_info_regression_with_random_state = lambda X, y: mutual_info_regression(X, y, random_state=42)

# SelectKBest with mutual_info_regression evaluates all features
selector = SelectKBest(score_func=mutual_info_regression_with_random_state, k='all')
selector.fit(X, y)

# Rank the features using Mutual Information Regression algorithm
top_features = get_top_k_features(selector=selector, feature_names=X.columns,
                                  k=20, algorithm="Mutual Information Regression")

# Reduce the dataset to the top 10 features
X_top = X[top_features]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_top, y, test_size=0.2, random_state=42)

# Train and fit random forest classification model based on feature selected
train_and_fit_model(X_train, X_test, y_train, y_test)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Feature Rankings using Mutual Information Regression:
          Feature     Score
461  Attribute462  0.854961
460  Attribute461  0.765855
459  Attribute460  0.756661
105  Attribute106  0.755056
104  Attribute105  0.742452
..            ...       ...
601  Attribute602  0.065317
287  Attribute288  0.063761
582  Attribute583  0.060540
374  Attribute375  0.058239
472  Attribute473  0.032411

[617 rows x 2 columns]


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Model Accuracy with Top 10 Features: 0.6621794871794872

Classification Report:
              precision    recall  f1-score   support

         1.0       0.60      0.65      0.62        66
         2.0       0.44      0.60      0.51        50
         3.0       0.79      0.84      0.81        62
         4.0       0.69      0.76      0.73        59
         5.0       0.49      0.30      0.37        64
         6.0       0.43      0.34      0.38        59
         7.0       0.60      0.75      0.67        53
         8.0       0.60      0.61      0.61        70
         9.0       0.54      0.70      0.61        56
        10.0       0.84      0.87      0.86        55
        11.0       0.89      0.87      0.88        78
        12.0       0.75      0.61      0.67        67
        13.0       0.83      0.83      0.83        58
        14.0       0.75      0.71      0.73        66
        15.0       0.73      0.74      0.73        54
        16.0       0.57      0.56      0.56        59


In [40]:
from sklearn.feature_selection import f_classif
# Step 7: Apply ANOVA F-value Classificaiton

# SelectKBest with f_classif evaluates all features
selector = SelectKBest(score_func=f_classif, k='all')
selector.fit(X, y)

# Rank the features using ANOVA F-value Classificaiton algorithm
top_features = get_top_k_features(selector=selector, feature_names=X.columns,
                                  k=20, algorithm="ANOVA F-value Classificaiton")

# Reduce the dataset to the top 10 features
X_top = X[top_features]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_top, y, test_size=0.2, random_state=42)

# Train and fit random forest classification model based on feature selected
train_and_fit_model(X_train, X_test, y_train, y_test)

Feature Rankings using ANOVA F-value Classificaiton:
          Feature        Score
418  Attribute419  3746.378688
417  Attribute418  3470.067861
416  Attribute417  2645.365726
394  Attribute395  2338.155556
415  Attribute416  1928.357789
..            ...          ...
346  Attribute347    17.850687
343  Attribute344    16.862710
345  Attribute346    16.335827
344  Attribute345    14.390236
472  Attribute473     4.741746

[617 rows x 2 columns]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Model Accuracy with Top 10 Features: 0.6948717948717948

Classification Report:
              precision    recall  f1-score   support

         1.0       0.77      0.83      0.80        66
         2.0       0.39      0.60      0.47        50
         3.0       0.82      0.79      0.80        62
         4.0       0.55      0.49      0.52        59
         5.0       0.31      0.25      0.28        64
         6.0       0.85      0.90      0.88        59
         7.0       0.59      0.75      0.66        53
         8.0       0.94      0.91      0.93        70
         9.0       0.47      0.52      0.49        56
        10.0       0.74      0.82      0.78        55
        11.0       0.82      0.79      0.81        78
        12.0       0.80      0.67      0.73        67
        13.0       0.77      0.76      0.77        58
        14.0       0.74      0.64      0.68        66
        15.0       0.62      0.69      0.65        54
        16.0       0.58      0.64      0.61        59


In [41]:
from sklearn.feature_selection import f_regression
# Step 8: Apply ANOVA F-value Regression

# SelectKBest with f_regression evaluates all features
selector = SelectKBest(score_func=f_regression, k='all')
selector.fit(X, y)

# Rank the features using ANOVA F-value Classificaiton algorithm
top_features = get_top_k_features(selector=selector, feature_names=X.columns,
                                  k=20, algorithm="ANOVA F-value Regression")

# Reduce the dataset to the top 10 features
X_top = X[top_features]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_top, y, test_size=0.2, random_state=42)

# Train and fit random forest classification model based on feature selected
train_and_fit_model(X_train, X_test, y_train, y_test)

Feature Rankings using ANOVA F-value Regression:
          Feature        Score
479  Attribute480  1167.843722
432  Attribute433   901.785579
429  Attribute430   741.821940
169  Attribute170   732.346687
168  Attribute169   702.084490
..            ...          ...
66    Attribute67     0.012371
565  Attribute566     0.003878
44    Attribute45     0.001380
582  Attribute583     0.000094
344  Attribute345     0.000028

[617 rows x 2 columns]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Model Accuracy with Top 10 Features: 0.5814102564102565

Classification Report:
              precision    recall  f1-score   support

         1.0       0.37      0.21      0.27        66
         2.0       0.24      0.34      0.28        50
         3.0       0.44      0.69      0.54        62
         4.0       0.31      0.47      0.37        59
         5.0       0.45      0.38      0.41        64
         6.0       0.61      0.66      0.63        59
         7.0       0.29      0.26      0.28        53
         8.0       0.36      0.29      0.32        70
         9.0       0.83      0.86      0.84        56
        10.0       0.35      0.45      0.40        55
        11.0       0.54      0.41      0.47        78
        12.0       0.69      0.81      0.74        67
        13.0       0.75      0.76      0.75        58
        14.0       0.71      0.68      0.70        66
        15.0       0.79      0.78      0.79        54
        16.0       0.52      0.49      0.50        59
